In [1]:
    import pandas as pd
    import numpy as np
    import requests
    from bs4 import BeautifulSoup
    import bs4

In [8]:
headers = {
   'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
}

In [9]:
base_url = 'https://www.transfermarkt.co.uk/'
region_suffix = 'wettbewerbe/'
region_url = base_url + region_suffix
regions_list = ['amerika', 'europa', 'asien', 'europa', 'afrika']

In [10]:
regions_list[0]

'amerika'

In [14]:
def all_pages_url(url):
    return [url + F'?ajax=yw1&page={u}' for u in range(1, 50)]

In [15]:
all_pages_url(region_url + regions_list[0])

['https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=1',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=2',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=3',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=4',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=5',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=6',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=7',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=8',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=9',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=10',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=11',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=12',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=yw1&page=13',
 'https://www.transfermarkt.co.uk/wettbewerbe/amerika?ajax=y

In [11]:
def get_league_name(row):
    return row.find('img', {'class': ""}).get("alt")
    pass

def get_league_url(row):
    return row.findAll('a')[1].get('href')

def get_country(row):
    return row.find('img', {'class': 'flaggenrahmen'}).get('alt')

def get_clubs_number(row):
    return row.findAll('td', {'class': 'zentriert'})[1].contents[0]

def get_players_number(row):
    return row.findAll('td', {'class': 'zentriert'})[2].contents[0]

def get_avg_age(row):
    return row.findAll('td', {'class': 'zentriert'})[3].contents[0]

def get_foreign_players_percentage(row):
    return row.findAll('td', {'class': 'zentriert'})[4].find('a').contents[0]

def get_total_value(row):
    return row.find('td', {'class': 'rechts'}).contents[0]

In [12]:
print(region_url + regions_list[0])
result = requests.get(region_url + regions_list[0], headers=headers)
soup = BeautifulSoup(result.content, 'html5lib')
table = soup.find(name='table', attrs={'class': 'items'})
table_body = table.find(name='tbody')
rows = table_body.findAll('tr', attrs={'class': ['odd', 'even']})
t = []
for r in rows:
    t.append({
        'league': get_league_name(r),
        'url': get_league_url(r),
        'country': get_country(r),
        'clubs': get_clubs_number(r),
        'players': get_players_number(r),
        'avg_age': get_avg_age(r),
        'foreing_players': get_foreign_players_percentage(r),
        'total_value': get_total_value(r),
    })

https://www.transfermarkt.co.uk/wettbewerbe/amerika


In [13]:
pd.DataFrame(t)

,avg_age,clubs,country,foreing_players,league,players,total_value,url
0,"26,4",20,Brazil,"10,1 %",Campeonato Brasileiro Série A,685,£721.08m,/campeonato-brasileiro-serie-a/startseite/wett...
1,"26,8",26,Argentina,"14,1 %",Primera División,666,£665.80m,/primera-division/startseite/wettbewerb/AR1N
2,"26,3",18,Mexico,"36,8 %",Liga MX Apertura,497,£548.91m,/liga-mx-apertura/startseite/wettbewerb/MEXA
3,"26,4",23,United States,"55,1 %",Major League Soccer,652,£463.41m,/major-league-soccer/startseite/wettbewerb/MLS1
4,"26,1",20,Colombia,"11,1 %",Liga Águila II,678,£252.92m,/liga-aguila-ii/startseite/wettbewerb/COL1
5,"26,1",20,Colombia,"11,1 %",Liga Águila I,678,£252.92m,/liga-aguila-i/startseite/wettbewerb/COLP
6,"26,6",12,Ecuador,"17,4 %",Serie A Primera Etapa,305,£126.45m,/serie-a-primera-etapa/startseite/wettbewerb/ECPE
7,"26,6",12,Ecuador,"17,4 %",Serie A Segunda Etapa,305,£126.45m,/serie-a-segunda-etapa/startseite/wettbewerb/ECSE
8,"25,9",16,Chile,"23,4 %",Campeonato Scotiabank Primera División,470,£123.92m,/campeonato-scotiabank-primera-division/starts...
9,"25,6",8,Colombia,"10,2 %",Fase Final Liga Águila I,266,£110.66m,/fase-final-liga-aguila-i/startseite/wettbewer...


In [18]:
data = pd.read_csv('leagues.csv', index_col=0)

In [19]:
data

,avg_age,clubs,country,foreing_players,league,players,total_value,url
0,"26,4",20,Brazil,"10,1 %",Campeonato Brasileiro Série A,685.000,£721.08m,/campeonato-brasileiro-serie-a/startseite/wett...
1,"26,8",26,Argentina,"14,1 %",Primera División,666.000,£665.80m,/primera-division/startseite/wettbewerb/AR1N
2,"26,3",18,Mexico,"36,8 %",Liga MX Apertura,497.000,£548.91m,/liga-mx-apertura/startseite/wettbewerb/MEXA
3,"26,4",23,United States,"55,1 %",Major League Soccer,652.000,£463.41m,/major-league-soccer/startseite/wettbewerb/MLS1
4,"26,1",20,Colombia,"11,1 %",Liga Águila II,678.000,£252.92m,/liga-aguila-ii/startseite/wettbewerb/COL1
5,"26,1",20,Colombia,"11,1 %",Liga Águila I,678.000,£252.92m,/liga-aguila-i/startseite/wettbewerb/COLP
6,"26,6",12,Ecuador,"17,4 %",Serie A Primera Etapa,305.000,£126.45m,/serie-a-primera-etapa/startseite/wettbewerb/ECPE
7,"26,6",12,Ecuador,"17,4 %",Serie A Segunda Etapa,305.000,£126.45m,/serie-a-segunda-etapa/startseite/wettbewerb/ECSE
8,"25,9",16,Chile,"23,4 %",Campeonato Scotiabank Primera División,470.000,£123.92m,/campeonato-scotiabank-primera-division/starts...
9,"25,6",8,Colombia,"10,2 %",Fase Final Liga Águila I,266.000,£110.66m,/fase-final-liga-aguila-i/startseite/wettbewer...
